In [1]:
import pygame
import random
import math
import numpy as np 

# 常量，屏幕宽高
WIDTH, HEIGHT = 800, 600

# 初始化操作
pygame.init()
pygame.mixer.init()
# 创建游戏窗口
screen = pygame.display.set_mode((WIDTH, HEIGHT))

# 设置游戏标题
pygame.display.set_caption('倒车入库')

pygame 2.3.0 (SDL 2.24.2, Python 3.9.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
help(pygame.transform.rotozoom)

Help on built-in function rotozoom in module pygame.transform:

rotozoom(...)
    rotozoom(surface, angle, scale) -> Surface
    filtered scale and rotation



In [3]:
# 添加系统时钟，用于设置帧的刷新
FPS = 10
clock = pygame.time.Clock()

class Wheel(pygame.sprite.Sprite):
    WHEEL_WIDTH, WHEEL_HEIGHT = 60, 60
    
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load('./image/steeringwheel.jpeg')
        self.rect = pygame.rect.Rect(WIDTH - self.WHEEL_WIDTH, 0, self.WHEEL_WIDTH, self.WHEEL_HEIGHT)
        self.image = pygame.transform.scale(self.image, (self.rect.width, self.rect.height))
        self.original_image = self.image
        
    def update(self, angle):
        self.image = pygame.transform.rotate(self.original_image, angle)
        self.rect = self.image.get_rect(center=self.rect.center)
        
class Car(pygame.sprite.Sprite):
    WHEEL_WIDTH, WHEEL_HEIGHT = 60, 60
    
    def __init__(self):
        super().__init__()
        self.image = pygame.image.load('./image/kart.jpeg')
        self.rect = pygame.rect.Rect(WIDTH - self.WHEEL_WIDTH, 100, self.WHEEL_WIDTH, self.WHEEL_HEIGHT)
        self.image = pygame.transform.scale(self.image, (self.rect.width, self.rect.height))
        self.original_image = self.image
        
    def update(self, angle):
        self.image = pygame.transform.rotate(self.original_image, angle)
        self.rect = self.image.get_rect(center=self.rect.center)

# 主角
class Hero(pygame.sprite.Sprite):
    GEAR2DIR = {'REVERSE': -1, 'DRIVE': 1, 'PARK': 0}
    ANGLE_MAX = math.pi / 4
    CAR_LENGTH, CAR_WIDTH, TIRE_DIST = 60, 40, 40
    
    def __init__(self, speed=10):
        super().__init__()  # 调用父类的初始化方法
        self.image = pygame.image.load('./image/kart.jpeg')
        self.rect = pygame.rect.Rect(0, 0, self.CAR_LENGTH, self.CAR_WIDTH)
        self.image = pygame.transform.scale(self.image, (self.rect.width, self.rect.height))
        self.original_image = self.image
        
        self.direction = pygame.math.Vector2(1, 0)
        
        self.rect.x, self.rect.y = 0, 100  # 主角初始化位置
        self.speed = speed
        self.car_angle = 0
        self.steer_angle = 0
        self.gear = 'DRIVE'
        sound = pygame.mixer.Sound('deja-vu.mp3')
        sound.play(-1, 0)

    def update(self, *args):
        keys = pygame.key.get_pressed()
        
        # accelerate or ralentir 
        if keys[pygame.K_a]:
            self.speed += 10
        if keys[pygame.K_r]:
            self.speed -= 10
        self.speed = np.clip(self.speed, 10, 50)
        
        # adjust angle
        if keys[pygame.K_UP]:
            self.steer_angle += math.pi / 24 # 7.5度
        if keys[pygame.K_DOWN]:
            self.steer_angle -= math.pi / 24
        self.steer_angle = np.clip(self.steer_angle, -self.ANGLE_MAX, self.ANGLE_MAX)
        
        
        # top-left是(0, 0), y是倒过来的
        if keys[pygame.K_LEFT]:
            self.gear = 'REVERSE'
            self.rect.x -= self.speed * math.cos(self.steer_angle + self.car_angle)
            self.rect.y += self.speed * math.sin(self.steer_angle + self.car_angle)
            if self.steer_angle:
                delta_angle = self.speed / (self.TIRE_DIST / math.sin(self.steer_angle))
                print(f"delta_angle is {-delta_angle}")
                self.rotate(-delta_angle)
                
        if keys[pygame.K_RIGHT]:
            self.gear = 'DRIVE'
            self.rect.x += self.speed * math.cos(self.steer_angle + self.car_angle)
            self.rect.y -= self.speed * math.sin(self.steer_angle + self.car_angle)
            if self.steer_angle:
                delta_angle = self.speed / (self.TIRE_DIST / math.sin(self.steer_angle))
                self.rotate(delta_angle)
                
        # make sure the whole car is inside the screen
        self.rect.x = np.clip(self.rect.x, 0, WIDTH - self.rect.width)
        self.rect.y = np.clip(self.rect.y, 0, HEIGHT - self.rect.height)
                
    def rotate(self, delta_angle):
        self.direction.rotate_ip(delta_angle / math.pi * 180)
        self.car_angle += delta_angle
        self.image = pygame.transform.rotate(self.original_image, self.car_angle / math.pi * 180)
        self.rect = self.image.get_rect(center=self.rect.center)

In [4]:
# 初始化精灵组
hero_sprite = pygame.sprite.Group()
wheel_sprite = pygame.sprite.Group()
car_sprite = pygame.sprite.Group()

# 定义人物
hero = Hero(4)
hero_sprite.add(hero)

# 定义方向盘
wheel = Wheel()
wheel_sprite.add(wheel)
car = Car()
car_sprite.add(car)



In [5]:
# 保持游戏运行状态(游戏循环）
while True:
    # ===========游戏帧的刷新===========
    clock.tick(FPS)

    # 检测事件
    for event in pygame.event.get():
        # 检测关闭按钮被点击的事件
        if event.type == pygame.QUIT:
            # 退出
            pygame.quit()
            exit()
    
    screen.fill((0,0,0))
    
    
    # draw contour of garage 
    pygame.draw.line(screen, (160, 160, 160), (0, 80), (60, 80))
    pygame.draw.line(screen, (160, 160, 160), (0, 160), (60, 160))
    pygame.draw.line(screen, (160, 160, 160), (300, 0), (300, 300))

    
    hero_sprite.update()
    hero_sprite.draw(screen)
    wheel_sprite.update(hero.steer_angle / math.pi * 180)
    wheel_sprite.draw(screen)
    car_sprite.update(hero.car_angle / math.pi * 180)
    car_sprite.draw(screen)
    
    pygame.display.update()

2023-04-16 10:41:47.211 Python[1259:26310] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit


delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.06470476127563018
delta_angle is 0.064

error: display Surface quit